<a href="https://colab.research.google.com/github/SARATH062005/cat_vs_dog/blob/main/cat_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
datasets, info = tfds.load('cats_vs_dogs', with_info=True , as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.YTYPX0_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, Jo

In [6]:
class_names=info.features['label'].names
class_names

['cat', 'dog']

In [11]:
 for i , example in enumerate (datasets['train']):
  image, label = example
  save_dir= './cats_vs_dogs/tain/{}'.format(class_names[label])
  os.makedirs(save_dir, exist_ok=True)
  filename= save_dir + "/" + "{}_{}.jpg".format(class_names[label],i)
  tf.keras.preprocessing.image.save_img(filename, image.numpy())

  #break

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [30]:
datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2,rotation_range=10,
                             width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1,shear_range=0.10 ,horizontal_flip=True)
train_generator= datagen.flow_from_directory('/content/cats_vs_dogs/tain',
                                             target_size=(150,150),
                                             batch_size=216,
                                             class_mode='binary',
                                             subset='training')

validation_generator= datagen.flow_from_directory('/content/cats_vs_dogs/tain',
                                             target_size=(150,150),
                                             batch_size=216,
                                             class_mode='binary',
                                             subset='validation')

Found 18611 images belonging to 2 classes.
Found 4651 images belonging to 2 classes.


In [31]:
#build CNN

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [32]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 74, 74, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 36, 36, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 17, 17, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,035,073 (72.61 MB)

 Trainable params: 19,034,625 (72.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history=model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.6005 - loss: 3.4349 - val_accuracy: 0.5343 - val_loss: 0.7028
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.6747 - loss: 0.7408 - val_accuracy: 0.5012 - val_loss: 1.3390
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.7431 - loss: 0.5342 - val_accuracy: 0.5025 - val_loss: 1.6432
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.7646 - loss: 0.4885 - val_accuracy: 0.5012 - val_loss: 3.4509
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.7731 - loss: 0.4735 - val_accuracy: 0.5083 - val_loss: 2.1272
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.7836 - loss: 0.4515 - val_accuracy: 0.5242 - val_loss: 1.9036
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.8030 - loss: 0.4273 - val_accuracy: 0.7312 - val_loss: 0.6679
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8039 - loss: 0.4226 - val_accuracy: 0.7394 - v